# Complete Journey Analysis - Full Working Notebook
## Smart Shoppers vs. Margin Killers

This is your complete analysis in one notebook. Once you're done, you can split it back out for MyST if needed.

---

## Setup & Imports

In [ ]:
import warnings
import os

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from completejourney_py import get_data

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.2f}'.format)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Setup complete!")

## 1. Load Data

In [ ]:
# Load all datasets
all_data = get_data()

transactions = all_data["transactions"]
demographics = all_data["demographics"]
coupons = all_data["coupons"]
coupon_redemptions = all_data["coupon_redemptions"]

# Quick summary
data_summary = pd.DataFrame({
    'Dataset': ['Transactions', 'Demographics', 'Coupon Redemptions', 'Coupons'],
    'Count': [
        f'{len(transactions):,}', 
        f'{len(demographics):,}', 
        f'{len(coupon_redemptions):,}', 
        f'{len(coupons):,}'
    ]
})
display(data_summary.set_index('Dataset'))

print(f"\n📅 Date range: {transactions['transaction_timestamp'].min()} to {transactions['transaction_timestamp'].max()}")

In [ ]:
# Peek at the data
print("TRANSACTIONS:")
print(f"Columns: {transactions.columns.tolist()}")
display(transactions.head())

print("\nDEMOGRAPHICS:")
display(demographics.head())
print(f"\nMissing values:\n{demographics.isnull().sum()}")

## 2. Calculate Discount Usage

In [ ]:
# Identify discount columns
discount_cols = ['retail_disc', 'coupon_disc', 'coupon_match_disc']
discount_cols = [c for c in discount_cols if c in transactions.columns]

if discount_cols:
    transactions['has_discount'] = (transactions[discount_cols] > 0).any(axis=1)
else:
    print("⚠️ No discount columns found")
    transactions['has_discount'] = False

# Overall discount rate
total_transactions = len(transactions)
discounted_transactions = transactions['has_discount'].sum()
discount_rate = discounted_transactions / total_transactions

print(f"Overall discount rate: {discount_rate:.1%}")
print(f"Transactions with discounts: {discounted_transactions:,} / {total_transactions:,}")

In [ ]:
# Calculate per-household discount usage
discount_usage = (
    transactions
    .groupby('household_id')
    .agg(
        total_items=('has_discount', 'size'),
        discounted_items=('has_discount', 'sum'),
        total_sales=('sales_value', 'sum')
    )
    .reset_index()
)

discount_usage['discount_share'] = (
    discount_usage['discounted_items'] / discount_usage['total_items']
)

print("\nDiscount share distribution:")
print(discount_usage['discount_share'].describe())

## 3. Calculate Coupon Usage

In [ ]:
# Count coupons per household
coupon_usage = (
    coupon_redemptions
    .groupby('household_id')
    .size()
    .rename('num_coupons_redeemed')
    .reset_index()
)

print(f"Households with coupons: {len(coupon_usage):,}")
print(f"Total coupons redeemed: {coupon_usage['num_coupons_redeemed'].sum():,}")
print(f"Average per household: {coupon_usage['num_coupons_redeemed'].mean():.1f}")

## 4. Calculate Spending Metrics

In [ ]:
# Find quantity column
qty_col = None
for candidate in ['quantity', 'QUANTITY', 'purchase_quantity']:
    if candidate in transactions.columns:
        qty_col = candidate
        break

# Aggregate spending by household
agg_dict = {
    'sales_value': 'sum',
    'basket_id': pd.Series.nunique,
}

if qty_col:
    agg_dict[qty_col] = 'sum'

household_spend = (
    transactions
    .groupby('household_id')
    .agg(agg_dict)
    .rename(columns={
        'sales_value': 'total_sales',
        'basket_id': 'num_trips',
    })
    .reset_index()
)

household_spend['avg_basket_value'] = (
    household_spend['total_sales'] / household_spend['num_trips']
)

if qty_col:
    household_spend.rename(columns={qty_col: 'total_quantity'}, inplace=True)
    household_spend['avg_price_per_unit'] = (
        household_spend['total_sales'] / household_spend['total_quantity'].replace(0, np.nan)
    )

print("Spending metrics calculated:")
print(household_spend[['total_sales', 'num_trips', 'avg_basket_value']].describe())

## 5. Merge Everything into Master Dataset

In [ ]:
# Create master household dataset
households = (
    household_spend
    .merge(discount_usage[['household_id', 'discount_share', 'discounted_items', 'total_items']], 
           on='household_id', how='left', suffixes=('', '_disc'))
    .merge(coupon_usage, on='household_id', how='left')
    .merge(demographics, on='household_id', how='left')
)

# Fill missing values
households['num_coupons_redeemed'] = households['num_coupons_redeemed'].fillna(0)
households['discount_share'] = households['discount_share'].fillna(0)

# Calculate coupons per 100 items
households['coupons_per_100_items'] = (
    households['num_coupons_redeemed'] / households['total_items'] * 100
)

print(f"✅ Master dataset created: {len(households):,} households")
print(f"\nColumns: {households.columns.tolist()}")

In [ ]:
# Preview the merged data
display(households[[
    'household_id', 'total_sales', 'avg_basket_value', 'discount_share', 
    'num_coupons_redeemed', 'income', 'kids_count', 'household_comp'
]].head(10))

## 6. Define Promotion Segments

In [ ]:
# Calculate thresholds
disc_threshold = households["discount_share"].quantile(0.75)
coupon_threshold = households["coupons_per_100_items"].quantile(0.50)
extreme_disc_threshold = households["discount_share"].quantile(0.90)
extreme_coupon_threshold = households["coupons_per_100_items"].quantile(0.90)

print("Promotion thresholds:")
print(f"  Discount share (75th %ile): {disc_threshold:.1%}")
print(f"  Discount share (90th %ile): {extreme_disc_threshold:.1%}")
print(f"  Coupon usage (50th %ile): {coupon_threshold:.2f}")
print(f"  Coupon usage (90th %ile): {extreme_coupon_threshold:.2f}")

In [ ]:
# Create flags
households["is_high_discount"] = households["discount_share"] >= disc_threshold
households["is_high_coupon"] = households["coupons_per_100_items"] >= coupon_threshold
households["is_extreme_disc"] = households["discount_share"] >= extreme_disc_threshold
households["is_extreme_coupon"] = households["coupons_per_100_items"] >= extreme_coupon_threshold

# Define promo-heavy: (high discount AND high coupon) OR extreme on either
households["promo_heavy_flag"] = (
    (households["is_high_discount"] & households["is_high_coupon"])
    | households["is_extreme_disc"]
    | households["is_extreme_coupon"]
)

households["promo_segment"] = np.where(
    households["promo_heavy_flag"], "promo_heavy", "promo_light"
)

# Show distribution
segment_counts = households["promo_segment"].value_counts()
print("\nPromotion segments:")
for segment, count in segment_counts.items():
    pct = count / len(households) * 100
    print(f"  {segment}: {count:,} ({pct:.1f}%)")

## 7. Define Spending Segments

In [ ]:
# High spend = top 25%
spend_threshold = households['total_sales'].quantile(0.75)

households['spend_segment'] = np.where(
    households['total_sales'] >= spend_threshold,
    'high_spend',
    'low_spend'
)

print(f"High-spend threshold (75th percentile): ${spend_threshold:.2f}")
print("\nSpend segment distribution:")
print(households['spend_segment'].value_counts())

## 8. Create 2×2 Shopper Type Matrix

In [ ]:
# Create shopper type
households['shopper_type'] = (
    households['promo_segment'] + ' & ' + households['spend_segment']
)

print("\n" + "="*80)
print("2×2 SHOPPER TYPE MATRIX")
print("="*80)

shopper_type_counts = households['shopper_type'].value_counts()
for shopper_type, count in shopper_type_counts.items():
    pct = count / len(households) * 100
    print(f"  {shopper_type}: {count:,} ({pct:.1f}%)")

In [ ]:
# Summary metrics by shopper type
shopper_summary = (
    households
    .groupby('shopper_type')
    .agg(
        num_households=('household_id', 'nunique'),
        mean_total_sales=('total_sales', 'mean'),
        mean_avg_basket=('avg_basket_value', 'mean'),
        mean_num_trips=('num_trips', 'mean'),
        mean_discount_share=('discount_share', 'mean'),
        mean_coupons_per_100=('coupons_per_100_items', 'mean')
    )
    .round(2)
)

if 'avg_price_per_unit' in households.columns:
    shopper_summary['mean_avg_price_unit'] = (
        households.groupby('shopper_type')['avg_price_per_unit'].mean().round(2)
    )

print("\nMetrics by shopper type:")
display(shopper_summary)

## 9. Analyze Smart Shoppers

In [ ]:
# Focus on smart shoppers
smart_shoppers = households['shopper_type'] == 'promo_heavy & high_spend'
num_smart = smart_shoppers.sum()
pct_smart = num_smart / len(households) * 100

print("\n" + "="*80)
print("SMART SHOPPERS ANALYSIS")
print("="*80)
print(f"\n🎯 Smart shoppers: {num_smart:,} households ({pct_smart:.1f}%)")

# Compare to everyone else
smart_vs_others = (
    households
    .assign(is_smart_shopper=smart_shoppers)
    .groupby('is_smart_shopper')
    .agg(
        num_households=('household_id', 'nunique'),
        mean_total_sales=('total_sales', 'mean'),
        mean_avg_basket=('avg_basket_value', 'mean'),
        mean_discount_share=('discount_share', 'mean'),
        mean_coupons_per_100=('coupons_per_100_items', 'mean')
    )
    .round(2)
)

if 'avg_price_per_unit' in households.columns:
    smart_vs_others['mean_avg_price_unit'] = (
        households
        .assign(is_smart_shopper=smart_shoppers)
        .groupby('is_smart_shopper')['avg_price_per_unit']
        .mean()
        .round(2)
    )

smart_vs_others.index = ['All Others', 'Smart Shoppers']
display(smart_vs_others)

In [ ]:
# Calculate lifts
smart_basket = smart_vs_others.loc['Smart Shoppers', 'mean_avg_basket']
others_basket = smart_vs_others.loc['All Others', 'mean_avg_basket']
basket_lift = ((smart_basket - others_basket) / others_basket) * 100

smart_sales = smart_vs_others.loc['Smart Shoppers', 'mean_total_sales']
others_sales = smart_vs_others.loc['All Others', 'mean_total_sales']
sales_lift = ((smart_sales - others_sales) / others_sales) * 100

print("\n📊 Smart Shoppers vs. Everyone Else:")
print(f"  Basket value lift: +{basket_lift:.1f}%")
print(f"  Annual spending lift: +{sales_lift:.1f}%")
print(f"  Discount usage: {smart_vs_others.loc['Smart Shoppers', 'mean_discount_share']:.1%}")

## 10. Visualizations

In [ ]:
# Shopper type distribution and basket values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Color map
colors_map = {
    "promo_heavy & high_spend": "#27ae60",
    "promo_heavy & low_spend": "#e74c3c",
    "promo_light & high_spend": "#f39c12",
    "promo_light & low_spend": "#95a5a6",
}

# Left: Distribution
ax1 = axes[0]
sorted_types = shopper_type_counts.sort_values(ascending=True)
colors = [colors_map.get(x, "#3498db") for x in sorted_types.index]

bars = ax1.barh(range(len(sorted_types)), sorted_types.values, color=colors)
ax1.set_yticks(range(len(sorted_types)))
ax1.set_yticklabels(sorted_types.index)
ax1.set_xlabel("Number of Households", fontsize=12, fontweight="bold")
ax1.set_title("Household Distribution: 2×2 Shopper Matrix", fontsize=13, fontweight="bold")
ax1.grid(axis="x", alpha=0.3)

for i, (bar, val) in enumerate(zip(bars, sorted_types.values)):
    pct = val / len(households) * 100
    ax1.text(val + 5, i, f"{val:,} ({pct:.1f}%)", va="center", fontsize=10, fontweight="bold")

# Right: Basket values
ax2 = axes[1]
basket_by_type = shopper_summary["mean_avg_basket"].sort_values(ascending=True)
colors2 = [colors_map.get(x, "#3498db") for x in basket_by_type.index]

bars2 = ax2.barh(range(len(basket_by_type)), basket_by_type.values, color=colors2)
ax2.set_yticks(range(len(basket_by_type)))
ax2.set_yticklabels(basket_by_type.index)
ax2.set_xlabel("Average Basket Value ($)", fontsize=12, fontweight="bold")
ax2.set_title("Basket Value by Shopper Type", fontsize=13, fontweight="bold")
ax2.grid(axis="x", alpha=0.3)

for i, (bar, val) in enumerate(zip(bars2, basket_by_type.values)):
    ax2.text(val + 0.5, i, f"${val:.2f}", va="center", fontsize=10, fontweight="bold")

plt.tight_layout()
plt.show()

## 11. Demographics of Smart Shoppers

In [ ]:
smart_shopper_households = households[smart_shoppers]

print("\n" + "="*80)
print("DEMOGRAPHIC PROFILE: SMART SHOPPERS")
print("="*80)

if 'income' in households.columns:
    print("\n💰 Income distribution:")
    smart_income = smart_shopper_households['income'].value_counts(dropna=True).sort_index()
    for income, count in smart_income.items():
        pct = count / len(smart_shopper_households) * 100
        print(f"  {income}: {count} ({pct:.1f}%)")
    print(f"  → Most common: {smart_income.idxmax()}")

if 'kids_count' in households.columns:
    print("\n👨‍👩‍👧‍👦 Kids:")
    print(f"  Average: {smart_shopper_households['kids_count'].mean():.1f}")
    
if 'age' in households.columns:
    print("\n🎂 Age distribution:")
    smart_age = smart_shopper_households['age'].value_counts(dropna=True).sort_index()
    for age, count in smart_age.items():
        pct = count / len(smart_shopper_households) * 100
        print(f"  {age}: {count} ({pct:.1f}%)")

## 12. Final Summary Stats

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY STATISTICS")
print("="*80)

print(f"\n📊 Overall Population:")
print(f"  Total households: {len(households):,}")
print(f"  Total transactions: {len(transactions):,}")
print(f"  Date range: {transactions['transaction_timestamp'].min().date()} to {transactions['transaction_timestamp'].max().date()}")

print(f"\n🎯 Key Segments:")
print(f"  Promo-heavy: {(households['promo_segment']=='promo_heavy').sum():,} ({(households['promo_segment']=='promo_heavy').sum()/len(households)*100:.1f}%)")
print(f"  High-spend: {(households['spend_segment']=='high_spend').sum():,} ({(households['spend_segment']=='high_spend').sum()/len(households)*100:.1f}%)")
print(f"  Smart shoppers: {num_smart:,} ({pct_smart:.1f}%)")

if num_smart > 0:
    print(f"\n💡 Smart Shopper Highlights:")
    print(f"  Avg basket: ${smart_basket:.2f} (vs ${others_basket:.2f} others)")
    print(f"  Avg annual spend: ${smart_sales:.2f} (vs ${others_sales:.2f} others)")
    print(f"  Discount usage: {smart_vs_others.loc['Smart Shoppers', 'mean_discount_share']:.1%}")

---
## ✅ Analysis Complete!

Now you can:
1. Use these results for your PowerPoint
2. Extract key visualizations
3. Calculate any additional metrics you need
4. Split back into separate notebooks if needed for MyST

All your variables are available in this single session!